# Map

## Goal: Practice Map Visualization

## Load Map Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt

In [ ]:
# Load data

df = pd.read_csv('/Users/lnagata/Documents/Python/Shared/Share/ap_loc_union_all_men.csv',
                usecols = ['month','sub_categ_nm','zip','city','state','pageviews'])

In [ ]:
df.dtypes

In [ ]:
df['zip']=df['zip'].str.strip()

In [ ]:
# did not work: df = df.astype({"zip": int})
# did not work: df[['zip']] = df[['zip']].apply(pd.to_numeric)
# deprecated df[['zip']] = df.convert_objects(convert_numeric=True)
# did not work: df[['zip']] = df.apply(lambda col:pd.to_numeric(col, errors='coerce'))
# df['zip'] = df['zip'].str.replace(';','').astype(int)
#df["zip"] = pd.to_numeric(df["zip"])



cols = df.columns.drop(['month','sub_categ_nm','city','state','pageviews'])


In [ ]:
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
df.dtypes

In [ ]:
df['zip'] = df['zip'].fillna(0.0).astype(int)

In [ ]:
df.dtypes

In [ ]:
# df[df['zip'].str.isalnum()]

In [ ]:
df

In [ ]:
# For a specific month, group by sub_categ_nm, zip, aggregate by sum pageviews

df[df['month']=='6/1/19'].groupby(['zip']).sum()

In [ ]:
len(df['zip'].unique())

In [ ]:
df2 = pd.read_csv('/Users/lnagata/Documents/Python/Shared/us-zip-code-latitude-and-longitude.csv',
                 usecols = ['Zip','City','State','Latitude','Longitude'],
                                  delimiter = ';')
df2.rename(columns = {'Zip':'zip'}, inplace = True) 

In [ ]:
df2.dtypes

In [ ]:
df_inner = pd.merge(df, df2, on='zip', how='left')
df_inner.drop(['City','State'],axis=1, inplace=True)

In [ ]:
df_inner.dtypes

In [ ]:
df_inner = df_inner[df_inner['zip']!=0]

In [ ]:
df_inner

In [ ]:
# For a specific month, group by sub_categ_nm, zip, aggregate by sum pageviews

df_inner[df_inner['month']=='6/1/19'].groupby(['sub_categ_nm','zip'])

In [ ]:
df_inner.dtypes

In [ ]:
df_inner

In [ ]:
df_inner2 = df_inner[df_inner['sub_categ_nm']=='Mens Sweaters']

In [ ]:
df_inner2

In [ ]:
len(df_inner2)

In [ ]:
df_inner

In [ ]:
alt.data_transformers.disable_max_rows()

alt.Chart(df_inner2).mark_circle(size=3).encode(
longitude='Longitude:Q',
latitude='Latitude:Q',
color='sub_categ_nm:N',
tooltip='zip:N'
).project(
    type='albersUsa'
).properties(
    width=650,
    height=400
)

In [ ]:
df_loc = pd.read_csv('/Users/lnagata/Documents/Python/Shared/locs.csv')

In [ ]:
df_loc

In [ ]:
alt.data_transformers.disable_max_rows()

alt.Chart(df_loc).mark_circle(size=3).encode(
longitude='Longitude Dgr:Q',
latitude='Latitude Dgr:Q',
tooltip='Postal Code:N'
).project(
    type='albersUsa'
).properties(
    width=650,
    height=400
).configure_mark(
    opacity=0.7,
    color='red'
)

In [ ]:
df_loc.rename(columns = {'Postal Code':'zip'}, inplace = True) 

In [ ]:
df_loc

In [ ]:
df3 = pd.merge(df_inner, df_loc, on='zip', how='left')

In [ ]:
df3

In [ ]:
 # How many cat zips are served by a subway, how many are not?

In [ ]:
df3['Served'] = df3['Target'].isna()== False

In [ ]:
df3

In [ ]:
df4 = df3[df3['Served']== True]

In [ ]:
df5 = df3[df3['Served']== False]

In [ ]:
(df4.groupby(['state']).count()/df3.groupby(['state']).count()).sort_values(by = 'Served', ascending = False).index

In [ ]:
count_order = list(df4.groupby(['state']).count().sort_values(by = 'Served', ascending = False).index)

In [ ]:
count_order2 = list((df4.groupby(['state']).count()/df3.groupby(['state']).count()).sort_values(by = 'Served', ascending = False).index)

In [ ]:
alt.Chart(df3).mark_bar().encode(
    x=alt.X('state', sort=count_order),
    y='count(Served)',
    color='Served'
)

In [ ]:
alt.Chart(df3).mark_bar().encode(
    x=alt.X('count(Served)', stack="normalize"),
    y=alt.Y('state', sort=count_order2),
    color='Served'
)

##### We know zip codes served by Target stores and if we know zip codes not served.
##### We have lat and long for every zip.
##### Can we assess where we need to add more stores?
##### We should set a distance threshold--how far can a store reach in radius? Let's set as 5 miles
##### If a zip code is not served within 5 miles of any Target store, we need to build a store at the under-served zip
##### If a zip code is within 5 miles of one or more stores, we need to assign the zip code to a specific store 
##### Assumption: geo irregularity is irrelevant; straight distance in miles or km is enough
##### Assumption: assume the zip code's lat and long is all we need. if a zip code covers a large spacial area is irrelevant

In [ ]:
# calculate all distances between every store vs every zip
# 

In [ ]:
df3.describe()

In [ ]:
import pandas_profiling

In [ ]:
pandas_profiling.ProfileReport(df3)